# Introduction

Unfortunately all my file submission code is built around submitting libraries attached to  experiments, and these 4 files are very different from that.

In [ ]:
import pandas
import os
import sys
import hashlib
import jsonschema
import requests
import logging
import importlib
import gzip

importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

In [ ]:
ROOT = os.path.expanduser('~/proj/htsworkflow')
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [ ]:
from htsworkflow.submission.encoded import ENCODED
from htsworkflow.submission import aws_submission

In [ ]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [ ]:
blacklist = [
    'http://woldlab.caltech.edu/~phe/temp2/hg19mitoblack.bed',
    'http://woldlab.caltech.edu/~phe/temp2/hg38mitoblack.bed',
    'http://woldlab.caltech.edu/~phe/temp2/mm9mitoblack.bed',
    'http://woldlab.caltech.edu/~phe/temp2/mm10mitoblack.bed',
]

metadata = {
    'dataset': '/annotations/ENCSR064IDX/',
    'file_format': 'bed',
    'file_format_type': 'bed3',
    'award': 'U54HG006998',
    'lab': 'barbara-wold',
    'output_type':  "semi-automated genome annotation",
}

In [ ]:
os.chdir('/dev/shm')

In [ ]:
def cache_blacklist(blacklist, metadata):
    for bed in blacklist:
        _, name = os.path.split(bed)
        name += '.gz'
        req = requests.get(bed)
        contents = req.content
        compressed = gzip.compress(contents, compresslevel=9)
        with open(name, 'wb') as outstream:
            outstream.write(compressed)
        payload = metadata.copy()
        payload['md5sum'] = hashlib.md5(compressed).hexdigest()
        payload['submitted_file_name'] = name
        yield payload

In [ ]:
for payload in cache_blacklist(blacklist, metadata):
    aws_submission.upload_file(server, payload, dry_run=False)